## scButterfly on perturbation datasets

Except PBMC dataset with one type of perturbation, we will explore datasets with more than one. Our goal would be to try scButterfly for two isolated groups of perturbation types, similar with the PBMC use case. Then we will attempt to create a multi task architecture, by leveraging the information of both of these two perturbations.

Our goal would be to have an evaluation pipeline as well, and to test the potential of scButterfly to different perturbation datasets than the PBMC one. Potentially comparing it with other models as well.

In [3]:
from pertpy import data
import scanpy as sc


sc.settings.datasetdir = "../data"

sciplex_gxe1 = data.sciplex_gxe1()
combosciplex = data.combosciplex()
srivatsan_2020_sciplex2 = data.srivatsan_2020_sciplex2()
srivatsan_2020_sciplex3 = data.srivatsan_2020_sciplex3()


In [4]:
sciplex_gxe1

AnnData object with n_obs × n_vars = 18585 × 58347
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell', 'sample', 'Size_Factor', 'n.umi', 'hash_umis_W', 'pval_W', 'qval_W', 'top_to_second_best_ratio_W', 'top_oligo_W', 'RTwell', 'Ligwell', 'PCR_well', 'PCR_plate', 'new_cell', 'CRISPR_hash', 'gRNA_library', 'dose', 'treatment', 'time_point', 'protospacer_sequence', 'gRNA_id', 'gene_id', 'CRISPR_gRNA', 'guide_number'
    var: 'id', 'gene_short_name'

In [5]:
# sci-plex 
sciplex_gxe1.obs.groupby(['sample','treatment', 'dose']).size()

sample  treatment     dose 
0       dmso          0.0      3115
        temozolomide  0.1      1458
                      0.5      1454
                      1.0      1367
                      5.0      1191
                      10.0      919
                      50.0      695
                      100.0     620
        thioguanine   0.1      1902
                      0.5      1594
                      1.0      1372
                      5.0       986
                      10.0      754
                      50.0      606
                      100.0     552
dtype: int64

In [6]:
combosciplex

AnnData object with n_obs × n_vars = 63378 × 27518
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type'
    var: 'id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Drug1_colors', 'Drug2_colors', 'Well_colors', 'condition_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pathway1_colors', 'pathway2_colors', 'pathway_colors', 'pca', 'rank_genes_groups', 'split_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [7]:
combosciplex.obs.groupby(['cell_type','condition', 'Drug1', 'Drug2']).size()

/tmp/ipykernel_1030982/2935270339.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  combosciplex.obs.groupby(['cell_type','condition', 'Drug1', 'Drug2']).size()


cell_type  condition                 Drug1         Drug2       
A549       Alvespimycin+Pirarubicin  Alvespimycin  Alvespimycin       0
                                                   Carmofur           0
                                                   Cediranib          0
                                                   Crizotinib         0
                                                   Curcumin           0
                                                                   ... 
           control+control           control       SRT2104            0
                                                   SRT3025            0
                                                   Sorafenib          0
                                                   Tanespimycin       0
                                                   control         1451
Length: 4608, dtype: int64

In [8]:
srivatsan_2020_sciplex2


AnnData object with n_obs × n_vars = 24262 × 58347
    obs: 'ncounts', 'hash_umis', 'pval_demultiplexing', 'qval_demultiplexing', 'top_to_second_best_ratio', 'top_oligo', 'perturbation', 'dose_value', 'well', 'celltype', 'cell_line', 'cancer', 'disease', 'tissue_type', 'organism', 'perturbation_type', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'chembl-ID'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [10]:
srivatsan_2020_sciplex2.obs.groupby(['celltype','perturbation']).size()

/tmp/ipykernel_1030982/3907014773.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  srivatsan_2020_sciplex2.obs.groupby(['celltype','perturbation']).size()


celltype                         perturbation
alveolar basal epithelial cells  BMS             4183
                                 Dex             8064
                                 Nutlin          5956
                                 SAHA            5530
                                 control          529
dtype: int64

In [11]:
srivatsan_2020_sciplex3

AnnData object with n_obs × n_vars = 799317 × 110983
    obs: 'ncounts', 'well', 'plate', 'cell_line', 'replicate', 'time', 'dose_value', 'pathway_level_1', 'pathway_level_2', 'perturbation', 'target', 'pathway', 'dose_unit', 'celltype', 'disease', 'cancer', 'tissue_type', 'organism', 'perturbation_type'
    var: 'ensembl_id'

In [13]:
srivatsan_2020_sciplex3.obs.groupby(['perturbation']).size()

/tmp/ipykernel_1030982/2308955426.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  srivatsan_2020_sciplex3.obs.groupby(['perturbation']).size()


perturbation
2-Methoxyestradiol (2-MeOE2)     5009
(+)-JQ1                          5731
A-366                            3864
ABT-737                          3119
AC480 (BMS-599626)               3758
                                ...  
XAV-939                          3723
YM155 (Sepantronium Bromide)     1007
ZM 447439                        4719
Zileuton                         4052
control                         17578
Length: 189, dtype: int64

In [14]:
srivatsan_2020_sciplex3.obs.groupby(['celltype']).size()

/tmp/ipykernel_1030982/1321741874.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  srivatsan_2020_sciplex3.obs.groupby(['celltype']).size()


celltype
None                                36522
alveolar basal epithelial cells    244281
lymphoblasts                       173652
mammary epithelial cells           344862
dtype: int64

In [15]:
srivatsan_2020_sciplex3.obs.groupby(['celltype', 'perturbation']).size()

/tmp/ipykernel_1030982/3942280489.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  srivatsan_2020_sciplex3.obs.groupby(['celltype', 'perturbation']).size()


celltype                  perturbation                
None                      2-Methoxyestradiol (2-MeOE2)       0
                          (+)-JQ1                            0
                          A-366                              0
                          ABT-737                            0
                          AC480 (BMS-599626)                 0
                                                          ... 
mammary epithelial cells  XAV-939                         1854
                          YM155 (Sepantronium Bromide)      82
                          ZM 447439                       1588
                          Zileuton                        1959
                          control                         7786
Length: 756, dtype: int64

In [ ]:
from scButterfly.train_model_perturb import Model
import torch
import torch.nn as nn

In [ ]:
RNA_input_dim = control_data.X.shape[1]
ATAC_input_dim = stimulate_data.X.shape[1]

R_kl_div = 1 / RNA_input_dim * 20
A_kl_div = 1 / ATAC_input_dim * 20
kl_div = R_kl_div + A_kl_div

dataset = ""

file_path = f"./saved_results/butterfly/perturb/{dataset}"

# split data



In [ ]:
model_perturb1 = Model(
    R_encoder_nlayer = 2, 
    A_encoder_nlayer = 2,
    R_decoder_nlayer = 2, 
    A_decoder_nlayer = 2,
    R_encoder_dim_list = [RNA_input_dim, 256, 128],
    A_encoder_dim_list = [ATAC_input_dim, 128, 128],
    R_decoder_dim_list = [128, 256, RNA_input_dim],
    A_decoder_dim_list = [128, 128, ATAC_input_dim],
    R_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    R_decoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_decoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    translator_embed_dim = 128, 
    translator_input_dim_r = 128,
    translator_input_dim_a = 128,
    translator_embed_act_list = [nn.LeakyReLU(), nn.LeakyReLU(), nn.LeakyReLU()],
    discriminator_nlayer = 1,
    discriminator_dim_list_R = [128],
    discriminator_dim_list_A = [128],
    discriminator_act_list = [nn.Sigmoid()],
    dropout_rate = 0.1,
    R_noise_rate = 0.5,
    A_noise_rate = 0.5,
    chrom_list = [],
    logging_path = file_path,
    RNA_data = control_data,
    ATAC_data = stimulate_data
)

In [ ]:
model_perturb1.train(
    R_encoder_lr = 0.001,
    A_encoder_lr = 0.001,
    R_decoder_lr = 0.001,
    A_decoder_lr = 0.001,
    R_translator_lr = 0.001,
    A_translator_lr = 0.001,
    translator_lr = 0.001,
    discriminator_lr = 0.005,
    R2R_pretrain_epoch = 100,
    A2A_pretrain_epoch = 100,
    lock_encoder_and_decoder = False,
    translator_epoch = 200,
    patience = 50,
    batch_size = 64,
    r_loss = nn.MSELoss(size_average=True),
    a_loss = nn.MSELoss(size_average=True),
    d_loss = nn.BCELoss(size_average=True),
    loss_weight = [1, 1, 1, R_kl_div, A_kl_div, kl_div],
    train_id_r = train_id_r,
    train_id_a = train_id_a,
    validation_id_r = validation_id_r, 
    validation_id_a = validation_id_a, 
    output_path = file_path,
    seed = 19193,
    kl_mean = True,
    R_pretrain_kl_warmup = 50,
    A_pretrain_kl_warmup = 50,
    translation_kl_warmup = 50,
    load_model = None,
    logging_path = file_path
)

In [ ]:
model_perturb1.test(
    test_id_r = test_id_r,
    test_id_a = test_id_a, 
    model_path = None,
    load_model = False,
    output_path = None,
    test_pca = True,
    test_DEGs = True,
    test_R2 = True,
    test_dotplot = True,
    output_data = False,
    return_predict = False
)

In [ ]:
RNA_input_dim = control_data.X.shape[1]
ATAC_input_dim = stimulate_data.X.shape[1]

R_kl_div = 1 / RNA_input_dim * 20
A_kl_div = 1 / ATAC_input_dim * 20
kl_div = R_kl_div + A_kl_div

dataset = ""

file_path = f"./saved_results/butterfly/perturb/{dataset}"

In [ ]:
model_perturb2 = Model(
    R_encoder_nlayer = 2, 
    A_encoder_nlayer = 2,
    R_decoder_nlayer = 2, 
    A_decoder_nlayer = 2,
    R_encoder_dim_list = [RNA_input_dim, 256, 128],
    A_encoder_dim_list = [ATAC_input_dim, 128, 128],
    R_decoder_dim_list = [128, 256, RNA_input_dim],
    A_decoder_dim_list = [128, 128, ATAC_input_dim],
    R_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    R_decoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_decoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    translator_embed_dim = 128, 
    translator_input_dim_r = 128,
    translator_input_dim_a = 128,
    translator_embed_act_list = [nn.LeakyReLU(), nn.LeakyReLU(), nn.LeakyReLU()],
    discriminator_nlayer = 1,
    discriminator_dim_list_R = [128],
    discriminator_dim_list_A = [128],
    discriminator_act_list = [nn.Sigmoid()],
    dropout_rate = 0.1,
    R_noise_rate = 0.5,
    A_noise_rate = 0.5,
    chrom_list = [],
    logging_path = file_path,
    RNA_data = Control_data,
    ATAC_data = Stimulate_data
)

In [ ]:
model_perturb2.train(
    R_encoder_lr = 0.001,
    A_encoder_lr = 0.001,
    R_decoder_lr = 0.001,
    A_decoder_lr = 0.001,
    R_translator_lr = 0.001,
    A_translator_lr = 0.001,
    translator_lr = 0.001,
    discriminator_lr = 0.005,
    R2R_pretrain_epoch = 100,
    A2A_pretrain_epoch = 100,
    lock_encoder_and_decoder = False,
    translator_epoch = 200,
    patience = 50,
    batch_size = 64,
    r_loss = nn.MSELoss(size_average=True),
    a_loss = nn.MSELoss(size_average=True),
    d_loss = nn.BCELoss(size_average=True),
    loss_weight = [1, 1, 1, R_kl_div, A_kl_div, kl_div],
    train_id_r = train_id_r,
    train_id_a = train_id_a,
    validation_id_r = validation_id_r, 
    validation_id_a = validation_id_a, 
    output_path = file_path,
    seed = 19193,
    kl_mean = True,
    R_pretrain_kl_warmup = 50,
    A_pretrain_kl_warmup = 50,
    translation_kl_warmup = 50,
    load_model = None,
    logging_path = file_path
)

In [ ]:
model_perturb2.test(
    test_id_r = test_id_r,
    test_id_a = test_id_a, 
    model_path = None,
    load_model = False,
    output_path = None,
    test_pca = True,
    test_DEGs = True,
    test_R2 = True,
    test_dotplot = True,
    output_data = False,
    return_predict = False
)